In [1]:
import rasterio
import rasterio.mask
import json
import os
import matplotlib.pyplot as plt 
import geopandas as gpd
import pandas as pd
import fiona 

from rasterstats import zonal_stats 
from rasterio.plot import show, show_hist

In [5]:

lst_stats = pd.read_json('stats/lst_stats.json')

In [27]:
lst_stats.head(10)


,id,mean,std,min,max
0,LE07_184030_19990930,23.800709,4.523431,11.268534,42.464803
1,LE07_184030_20010919,0.460359,15.906680,-23.509819,41.941846
2,LT05_184030_19990415,21.376584,6.076947,-3.319575,43.257784
3,LE07_184030_20000527,31.675817,6.584740,13.076667,63.147242
4,LE07_184030_20000612,30.259851,6.134779,17.540601,59.055872
5,LE07_184030_20000628,29.195076,7.260053,11.760729,55.326812
6,LT05_184030_20000128,-5.675374,2.864131,-16.496042,7.744556
7,LT05_184030_20061027,18.077105,2.906100,9.306591,32.039842
8,LT05_184030_20070928,24.174974,3.739041,8.021415,41.237734
9,LT05_184030_20060504,20.010414,5.110809,-1.422574,40.386647


In [26]:
lst_stats.columns

Index(['id', 'mean', 'std', 'min', 'max'], dtype='object')

In [25]:
# lets get rid of the index and name the index column 'id'
lst_stats.reset_index(inplace=True)
lst_stats.rename(columns={'index':'id'}, inplace=True)


In [9]:
id_df = pd.read_json('./data/id_list.json')

In [ ]:
#

In [17]:
from datetime import datetime

# lets add date and sort by it 
id_df['date'] = id_df['id'].apply(lambda x: datetime.strptime(x.split('_')[-1], "%Y%m%d"))
id_df = id_df[['id', 'collection', 'tier', 'season', 'date', 'ghs']]

In [18]:
id_df = id_df.sort_values(by='date')

In [20]:
id_df.head(10)

,id,collection,tier,season,date,ghs
2,LT05_184030_19990415,LT05,T1,spring,1999-04-15,JRC/GHSL/P2023A/GHS_POP/2000
0,LE07_184030_19990930,LE07,T1,autumn,1999-09-30,JRC/GHSL/P2023A/GHS_POP/2000
6,LT05_184030_20000128,LT05,T2,winter,2000-01-28,JRC/GHSL/P2023A/GHS_POP/2000
3,LE07_184030_20000527,LE07,T1,spring,2000-05-27,JRC/GHSL/P2023A/GHS_POP/2000
4,LE07_184030_20000612,LE07,T1,summer,2000-06-12,JRC/GHSL/P2023A/GHS_POP/2000
5,LE07_184030_20000628,LE07,T1,summer,2000-06-28,JRC/GHSL/P2023A/GHS_POP/2000
1,LE07_184030_20010919,LE07,T1,autumn,2001-09-19,JRC/GHSL/P2023A/GHS_POP/2000
13,LT05_184030_20050210,LT05,T1,winter,2005-02-10,JRC/GHSL/P2023A/GHS_POP/2005
9,LT05_184030_20060504,LT05,T1,spring,2006-05-04,JRC/GHSL/P2023A/GHS_POP/2005
10,LT05_184030_20060520,LT05,T1,spring,2006-05-20,JRC/GHSL/P2023A/GHS_POP/2005


In [30]:
# now lets join the two dataframes 
lst_stats = lst_stats.merge(id_df, on='id')

In [31]:
lst_stats.sort_values(by='date', inplace=True)

In [32]:
lst_stats

,id,mean,std,min,max,collection,tier,season,date,ghs
2,LT05_184030_19990415,21.376584,6.076947,-3.319575,43.257784,LT05,T1,spring,1999-04-15,JRC/GHSL/P2023A/GHS_POP/2000
0,LE07_184030_19990930,23.800709,4.523431,11.268534,42.464803,LE07,T1,autumn,1999-09-30,JRC/GHSL/P2023A/GHS_POP/2000
6,LT05_184030_20000128,-5.675374,2.864131,-16.496042,7.744556,LT05,T2,winter,2000-01-28,JRC/GHSL/P2023A/GHS_POP/2000
3,LE07_184030_20000527,31.675817,6.584740,13.076667,63.147242,LE07,T1,spring,2000-05-27,JRC/GHSL/P2023A/GHS_POP/2000
4,LE07_184030_20000612,30.259851,6.134779,17.540601,59.055872,LE07,T1,summer,2000-06-12,JRC/GHSL/P2023A/GHS_POP/2000
5,LE07_184030_20000628,29.195076,7.260053,11.760729,55.326812,LE07,T1,summer,2000-06-28,JRC/GHSL/P2023A/GHS_POP/2000
1,LE07_184030_20010919,0.460359,15.906680,-23.509819,41.941846,LE07,T1,autumn,2001-09-19,JRC/GHSL/P2023A/GHS_POP/2000
13,LT05_184030_20050210,-8.998039,2.396483,-18.526346,6.784092,LT05,T1,winter,2005-02-10,JRC/GHSL/P2023A/GHS_POP/2005
9,LT05_184030_20060504,20.010414,5.110809,-1.422574,40.386647,LT05,T1,spring,2006-05-04,JRC/GHSL/P2023A/GHS_POP/2005
10,LT05_184030_20060520,30.101013,5.477231,3.571153,57.750188,LT05,T1,spring,2006-05-20,JRC/GHSL/P2023A/GHS_POP/2005


In [48]:
# lets group by season see trends for mean lst 

lst_stats.groupby('season')['mean'].mean()

season
autumn    20.339776
spring    24.804808
summer    30.928654
winter    -0.628140
Name: mean, dtype: float64